## Configure Path

In [1]:
import pathlib

DATASET_PATH = pathlib.Path('dataset.pkl')
if not(DATASET_PATH.exists()) :
    DATASET_PATH.mkdir()

## Create dataset

In [2]:
import pathlib
import pandas
import numpy

def add(
    dataset: pandas.DataFrame,
    image: pathlib.Path,
    alpha: float,
    beta: float,
    sigma: float,
    tol: float = 10**(-4),
    iterations: int = 300,
    blur: tuple[int, float] = numpy.nan,
    noise: float = numpy.nan
) -> pandas.DataFrame:

    to_add = pandas.DataFrame(
        {
            'image' : [image], 
            'alpha' : [alpha], 
            'beta' : [beta], 
            'sigma' : [sigma], 
            'tol' : [tol], 
            'iterations': [iterations], 
            'blur': [blur], 
            'noise': [noise]
        }
    )

    return pandas.concat([dataset, to_add], ignore_index=True)

In [3]:
dataset = pandas.DataFrame(columns=['image' , 'alpha', 'beta', 'sigma', 'tol', 'iterations', 'blur', 'noise'])
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, (3, 3), numpy.nan)
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, (7, 3), numpy.nan)
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, (15, 3), numpy.nan)
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, numpy.nan, 30)
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, numpy.nan, 20)
dataset = add(dataset, pathlib.Path('./inputs/Baboon.bmp'), 100, 1, 2, 10**(-4), 300, numpy.nan, 10)

In [4]:
pandas.to_pickle(dataset, DATASET_PATH)
dataset

,image,alpha,beta,sigma,tol,iterations,blur,noise
0,inputs/Baboon.bmp,100,1,2,0.0001,300,"(3, 3)",NaN
1,inputs/Baboon.bmp,100,1,2,0.0001,300,"(7, 3)",NaN
2,inputs/Baboon.bmp,100,1,2,0.0001,300,"(15, 3)",NaN
3,inputs/Baboon.bmp,100,1,2,0.0001,300,NaN,30
4,inputs/Baboon.bmp,100,1,2,0.0001,300,NaN,20
5,inputs/Baboon.bmp,100,1,2,0.0001,300,NaN,10


In [5]:
import sys
sys.path.append('../../..')

import lasp.io
import lasp.filters.linear
import lasp.noise

import scipy.signal

def image_from(params: pandas.Series) -> numpy.ndarray:

    image_path = params['image']
    # alpha = params['alpha']
    # beta = params['beta']
    # sigma = params['sigma']
    # tol = params['tol']
    # iterations = params['iterations']
    blur = params['blur']
    noise = params['noise']


    img = lasp.io.read(image_path)
    out = numpy.copy(img)

    if blur != numpy.nan:
        kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1])
        out = scipy.signal.convolve2d(out, kernel, mode='same')

    if noise != numpy.nan:
        out = lasp.noise.awgn(out, snr=noise)

    return out

def images_from(params: pandas.Series) -> tuple[numpy.ndarray, numpy.ndarray]:

    image_path = params['image']
    # alpha = params['alpha']
    # beta = params['beta']
    # sigma = params['sigma']
    # tol = params['tol']
    # iterations = params['iterations']
    blur = params['blur']
    noise = params['noise']


    img = lasp.io.read(image_path)
    out = numpy.copy(img)

    if blur != numpy.nan:
        kernel = lasp.filters.linear.gaussian_filter(size=blur[0], sigma=blur[1])
        out = scipy.signal.convolve2d(out, kernel, mode='same')

    if noise != numpy.nan:
        out = lasp.noise.awgn(out, snr=noise)

    return img, out

In [ ]:
DATASET_SIZE = dataset.index

for i in range(0, DATASET_SIZE):

    params = dataset.iloc[i]
    img, out = images_from(params)

    alpha = params['alpha']
    beta = params['beta']
    sigma = params['sigma']
    tol = params['tol']
    iterations = params['iterations']

    